The Python interpreter is a stack machine.

The stack provides a clean way to keep track of the state of the interpreter.



In [3]:
class Interpreter:
    def __init__(self):
        self.stack = []

    def LOAD_VALUE(self, number):
        self.stack.append(number)

    def PRINT_ANSWER(self):
        answer = self.stack.pop()
        print(answer)

    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)
        
    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        numbers = what_to_execute["numbers"]
        for each_step in instructions:
            instruction, argument = each_step
            if instruction == "LOAD_VALUE":
                number = numbers[argument]
                self.LOAD_VALUE(number)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()

what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),  # the first number
                     ("LOAD_VALUE", 1),  # the second number
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5] }

what_to_execute2 = {
    "instructions": [("LOAD_VALUE", 0),
                     ("LOAD_VALUE", 1),
                     ("ADD_TWO_VALUES", None),
                     ("LOAD_VALUE", 2),
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5, 8] }

interpreter = Interpreter()

interpreter.run_code(what_to_execute)
interpreter.run_code(what_to_execute2)

12
20


## Add Variables to our interpreter

### 2 features:
- Make use of Python's dynamic method lookup. We'll always define a method called FOO to execute the instruction called FOO, so we can use Python's getattr function to look up the method on the fly instead of using the big if statement.
- Parse argument based on instruction type

In [10]:
class Interpreter:
    def __init__(self):
        self.stack = []
        self.environment = {}
    
    def LOAD_VALUE(self, number):
        self.stack.append(number)
        
    def PRINT_ANSWER(self):
        answer = self.stack.pop()
        print(answer)

    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)
        
    def STORE_NAME(self, name):
        val = self.stack.pop()
        self.environment[name] = val

    def LOAD_NAME(self, name):
        val = self.environment[name]
        self.stack.append(val)

    def parse_argument(self, instruction, argument, what_to_execute):
        """ Understand what the argument to each instruction means."""
        numbers = ["LOAD_VALUE"]
        names = ["LOAD_NAME", "STORE_NAME"]

        if instruction in numbers:
            argument = what_to_execute["numbers"][argument]
        elif instruction in names:
            argument = what_to_execute["names"][argument]

        return argument

#     def run_code(self, what_to_execute):
#         instructions = what_to_execute["instructions"]
#         for each_step in instructions:
#             instruction, argument = each_step
#             argument = self.parse_argument(instruction, argument, what_to_execute)

#             if instruction == "LOAD_VALUE":
#                 self.LOAD_VALUE(argument)
#             elif instruction == "ADD_TWO_VALUES":
#                 self.ADD_TWO_VALUES()
#             elif instruction == "PRINT_ANSWER":
#                 self.PRINT_ANSWER()
#             elif instruction == "STORE_NAME":
#                 self.STORE_NAME(argument)
#             elif instruction == "LOAD_NAME":
#                 self.LOAD_NAME(argument)
                
    def execute(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)
            bytecode_method = getattr(self, instruction)
            if argument is None:
                bytecode_method()
            else:
                bytecode_method(argument)

"""
s() is compiled to following instructions.

>>> def s():
...     a = 1
...     b = 2
...     print(a + b)
"""
what_to_execute = {
        "instructions": [("LOAD_VALUE", 0),
                         ("STORE_NAME", 0),
                         ("LOAD_VALUE", 1),
                         ("STORE_NAME", 1),
                         ("LOAD_NAME", 0),
                         ("LOAD_NAME", 1),
                         ("ADD_TWO_VALUES", None),
                         ("PRINT_ANSWER", None)],
        "numbers": [1, 5],
        "names":   ["a", "b"] }

interpreter = Interpreter()

interpreter.execute(what_to_execute)

6
